In [44]:
import time

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal

# using scipy's 2d convolution function
conv2 = scipy.signal.convolve2d # modes include "full", "valid", and "same"

In [2]:
topten = [2, 1, 13, 12, 38, 10, 4, 5, 25, 9]
index_map = {}
for i, sign in enumerate(topten):
    index_map[sign] = i

In [3]:
loaded = np.load('/tmp/lenet5.npz') 
x0=loaded['x0']
x1=loaded['x1'] 
x1p=loaded['x1p']
x2=loaded['x2'] 
x2p=loaded['x2p']
x3=loaded['x3']
x3p=loaded['x3p']
epsinit=loaded['epsinit']
eta=loaded['eta'] 
n0=loaded['n0']
n1=loaded['n1']
n2=loaded['n2'] 
n3=loaded['n3'] 
w1=loaded['w1'] 
w2=loaded['w2'] 
w3=loaded['w3']
b1=loaded['b1']
b1p=loaded['b1p']
b2=loaded['b2']
b2p=loaded['b2p'] 
b3=loaded['b3'] 
b3p=loaded['b3p'] 
n4=loaded['n4']
n5=loaded['n5'] 
n6=loaded['n6'] 
W4=loaded['W4']
W5=loaded['W5']
W6=loaded['W6']
b4=loaded['b4'] 
b5=loaded['b5'] 
b6=loaded['b6']

In [4]:
x0.shape

(3, 40, 40)

In [5]:
W4.shape

(160, 180)

In [6]:
def f(x): # hyperbolic tangent activation function
    return np.tanh(x)
def df(y): # derivative of f composed with inverse of f
    return 1 - np.multiply(y, y)

def maxpool(images):
    # 2x2 max pooling for set of images
    # returns MAXIMA, WINNERS
    # MAXIMA 2x smaller image, maximum of each 2x2 patch
    # WINNERS 2x smaller image, argmax (1..4) of each 2x2 patch

    [k, m, n] = images.shape  # mxn images, k of them
    # reshape to k x 2 x m/2 x 2 x n/2
    #print(m/2)
    #print(n/2)
    im_re = np.reshape(images, (k, m/2, 2, n/2, 2))
    maxima = np.amax(np.amax(im_re, 4), 2)
    
    # obtain "flat" indices for each element
    inds = np.reshape(np.array(range(im_re.size)), im_re.shape)
    
    # run two-step argmax to determine winning indices
    curr_inds = np.argmax(im_re, 4)
    dim0, dim1, dim2, dim3 = np.indices(curr_inds.shape)
    inds = inds[dim0, dim1, dim2, dim3, curr_inds]
    im_re = np.amax(im_re, 4)
    curr_inds = np.argmax(im_re, 2)
    dim0, dim1, dim3 = np.indices(curr_inds.shape)
    winners = inds[dim0, dim1, curr_inds, dim3]
    return maxima, winners

In [7]:
def convert(sample):
    return np.array([sample[:,:,0],sample[:,:,1],sample[:,:,2]])

In [8]:
def forward_pass(sample):
    global epsinit, eta, n0, n1, n2, n3, w1, w2, w3
    global b1, b1p, b2, b2p, b3, b3p, n4, n5, n6, W4, W5, W6, b4, b5, b6
    global x0, x1, x1p, x2, x2p, x3, x3p

    x0 = convert(sample)

    x1 = np.zeros(x1.shape)   # valid convolution by w1 reduces image size by 2
    x1p = np.zeros(x1p.shape)  # pooling reduces image size by 2x
    x2 = np.zeros(x2.shape)   # convolution by w2 reduces image size by 3
    x2p = np.zeros(x2p.shape)    # pooling reduces image size by 2x
    x3 = np.zeros(x3.shape)     # convolution by w3 reduces image size by 2
    x3p = np.zeros(x3p.shape)    # pooling reduces image size by 2x

    for i in range(n1):
        for j in range(n0):
            x1[i,:,:] += conv2(x0[j,:,:], w1[i,j,:,:], "valid")

    for i in range(n1):
        x1[i,:,:] = x1[i,:,:] + b1[i]

    x1 = f(x1)
    #print(x1.shape)

    #x1p, x1w = maxpool(x1)

    for i in range(n1):
        x1p[i,:,:] = x1p[i,:,:] + b1p[i]

    x1p, x1w = maxpool(x1)

    for i in range(n2):
        for j in range(n1):
            x2[i,:,:] += conv2(x1p[j,:,:], w2[i,j,:,:], "valid")

    for i in range(n2):
        x2[i,:,:] = x2[i,:,:] + b2[i]

    x2 = f(x2)

    #x2p, x2w = maxpool(x2)

    for i in range(n2):
        x2p[i,:,:] = x2p[i,:,:] + b2p[i]

    x2p, x2w = maxpool(x2)

    for i in range(n3):
        for j in range(n2):
            x3[i,:,:] += conv2(x2p[j,:,:], w3[i,j,:,:], "valid")

    for i in range(n3):
        x3[i,:,:] = x3[i,:,:] + b3[i]

    x3 = f(x3)

    x3p, x3w = maxpool(x3)

    for i in range(n3):
        x3p[i,:,:] = x3p[i,:,:] + b3p[i]

    # discard 2D organization of x3p by reshaping to x3p(:)
    x4 = f(np.dot(W4,np.reshape(x3p,-1)) + b4)
    x5 = f(np.dot(W5,x4) + b5)
    x6 = f(np.dot(W6,x5) + b6)
    #x6 = f(np.dot(W6,x5) + b6)
    prediction = np.argmax(x6)
    
    return topten[prediction]
     

In [9]:
loaded = np.load('/tmp/classification.npz')
positives = loaded['positives']
pos_label = loaded['pos_label']

In [23]:
n = np.random.randint(len(positives) + 1)
print("Actual Label: ")
print(pos_label[n])
plt.imshow(positives[n])
print("Prediction: ")
print(forward_pass(positives[n]))

Actual Label: 
1
Prediction: 
1


In [29]:
#shuffle data
positives = np.array(positives)
pos_label = np.array(pos_label)
indexes = [i for i in range(0,len(positives))]
np.random.shuffle(indexes)
positives = positives[indexes]
pos_label = pos_label[indexes]

In [30]:
positives.shape

(18971, 40, 40, 3)

In [62]:
start_time = time.time()
predictions = []
for sample in positives[:5000]:
    predictions.append(forward_pass(sample))
print("--- %s seconds ---" % (time.time() - start_time))

--- 163.757057905 seconds ---


In [46]:
count = 0
for i, prediction in enumerate(predictions):
    if prediction != pos_label[i]:
        count += 1

print("Total Classification Error: " + str(float(count) / float(len(predictions))))

Total Classification Error: 0.0292


In [60]:
def error(class_):
    err_count = 0
    count = 0
    for i, prediction in enumerate(predictions):
        if pos_label[i] == class_:
            count += 1
            if prediction != pos_label[i]:
                err_count += 1
    return float(err_count) / float(count)

In [61]:
for class_ in topten:
    print("Classification Error For Class " + str(class_) + ": " + "{0:.5f}".format(error(class_)))

Classification Error For Class 2: 0.04787
Classification Error For Class 1: 0.11881
Classification Error For Class 13: 0.00350
Classification Error For Class 12: 0.00943
Classification Error For Class 38: 0.00318
Classification Error For Class 10: 0.00758
Classification Error For Class 4: 0.02087
Classification Error For Class 5: 0.03650
Classification Error For Class 25: 0.00000
Classification Error For Class 9: 0.02448
